# Conditional variational autoencoder
## Load Json Database of recipes

In [6]:
import json
import sys
sys.path.append('../..')

from utils import validateJson

jsonData = json.load(open('recipes_valid.json'))
#validateJson.validateRecimeJson(jsonData)

print(jsonData.keys())

dict_keys(['title', 'ingredients', 'instructions'])


## Convert list of ingredients to pandas dataframe and one hot encode the dataframe.

In [29]:
import pandas as pd

MAX_INGREDIENTS = 20

columns = []
data = []

for number, ingredient in enumerate(jsonData['ingredients'].values()):
    rowData = {}
    for key in ingredient.keys():
        for i, item in enumerate(ingredient[key].values()):
            rowData[key+str(i+1)] = item
    data.append(rowData)
    if number == 100:
        break

frame = pd.DataFrame(data)

frameEncoded = pd.get_dummies(frame)

In [101]:
from sklearn.preprocessing import OneHotEncoder


dropColumns = []
for column in frame.columns:
    if 'amount' in column:
        dropColumns.append(column)
frameStripped = frame.drop(columns=dropColumns)

enc = OneHotEncoder()
enc.fit(frameStripped)
frameStrippedEncodedSki = pd.DataFrame(data=enc.transform(frameStripped).toarray(), columns=enc.get_feature_names(frameStripped.columns))

frameEncodedSki = pd.concat([frame[dropColumns], frameStrippedEncodedSki], axis=1)
frameEncodedSki

,amount1,amount2,amount3,amount4,amount5,amount6,amount7,amount8,amount9,amount10,...,ingredient17_nan,unit18_,unit18_nan,unit19_tablespoon,unit19_nan,ingredient18_Belgian endive,ingredient18_salt,ingredient18_nan,ingredient19_fresh chives,ingredient19_nan
0,1.0,1.0,0.5,1.0,1.0,0.5,1.0,0.75,0.5,NaN,...,1.0,0.0,1.0,0.0,1.0,0.0,0.0,1.0,0.0,1.0
1,8.0,1.0,1.0,1.0,1.0,1.0,0.5,1.00,3.0,NaN,...,1.0,0.0,1.0,0.0,1.0,0.0,0.0,1.0,0.0,1.0
2,1.0,1.0,14.0,1.0,1.0,1.0,1.0,NaN,NaN,NaN,...,1.0,0.0,1.0,0.0,1.0,0.0,0.0,1.0,0.0,1.0
3,1.0,14.0,1.0,2.0,1.0,NaN,NaN,NaN,NaN,NaN,...,1.0,0.0,1.0,0.0,1.0,0.0,0.0,1.0,0.0,1.0
4,12.0,1.0,1.5,1.0,1.0,1.0,2.0,1.00,1.0,12.0,...,1.0,0.0,1.0,0.0,1.0,0.0,0.0,1.0,0.0,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
96,1.0,1.0,1.0,1.0,2.0,1.0,1.0,1.00,1.0,NaN,...,1.0,0.0,1.0,0.0,1.0,0.0,0.0,1.0,0.0,1.0
97,10.0,1.0,1.0,2.0,4.0,1.0,1.0,NaN,NaN,NaN,...,1.0,0.0,1.0,0.0,1.0,0.0,0.0,1.0,0.0,1.0
98,2.0,1.0,2.0,1.0,1.0,1.0,1.0,NaN,NaN,NaN,...,1.0,0.0,1.0,0.0,1.0,0.0,0.0,1.0,0.0,1.0
99,1.0,2.0,1.0,14.0,1.0,1.0,12.0,12.00,NaN,NaN,...,1.0,0.0,1.0,0.0,1.0,0.0,0.0,1.0,0.0,1.0


## VAE

In [102]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch import nn, optim
from torch.autograd import Variable

import pandas as pd
import numpy as np
from sklearn import preprocessing
from sklearn.model_selection import train_test_split


In [103]:
device = torch.device('cpu')
device

device(type='cpu')

In [104]:
def standardizeData(df):
    # replace nan with -99
    df = df.fillna(-99)
    # make all values float
    df = df.values.reshape(-1, df.shape[1]).astype('float32')
    # split randomly
    X_train, X_test = train_test_split(df, test_size=0.3, random_state=42)
    # standardize values
    scaler = preprocessing.StandardScaler()
    X_train = scaler.fit_transform(X_train)
    X_test = scaler.transform(X_test)

    return X_train, X_test, scaler


In [105]:
from torch.utils.data import Dataset, DataLoader

class DataBuilder(Dataset):
    def __init__(self, df, train=True):
        self.X_train, self.X_test, self.standardizer = standardizeData(df)
        if train:
            self.x = torch.from_numpy(self.X_train)
            self.len=self.x.shape[0]
        else:
            self.x = torch.from_numpy(self.X_test)
            self.len=self.x.shape[0]
        del self.X_train
        del self.X_test 
    def __getitem__(self,index):      
        return self.x[index]
    def __len__(self):
        return self.len

In [106]:
traindata_set=DataBuilder(frameEncodedSki, train=True)
testdata_set=DataBuilder(frameEncodedSki, train=False)

trainloader=DataLoader(dataset=traindata_set,batch_size=25)
testloader=DataLoader(dataset=testdata_set,batch_size=25)

In [107]:
type(trainloader.dataset.x), type(testloader.dataset.x)

(torch.Tensor, torch.Tensor)

In [108]:
trainloader.dataset.x.shape, testloader.dataset.x.shape

(torch.Size([70, 915]), torch.Size([31, 915]))

In [109]:
trainloader.dataset.x

tensor([[-0.3538, -0.4092, -0.4381,  ...,  0.1715, -0.1204,  0.1204],
        [-0.3538, -0.4092, -0.1431,  ...,  0.1715, -0.1204,  0.1204],
        [-0.1839, -0.4092, -0.3791,  ...,  0.1715, -0.1204,  0.1204],
        ...,
        [-0.3538, -0.4092, -0.3791,  ...,  0.1715, -0.1204,  0.1204],
        [-0.3538,  1.6071, -0.3791,  ...,  0.1715, -0.1204,  0.1204],
        [-0.4813, -0.5008, -0.3791,  ...,  0.1715, -0.1204,  0.1204]])

In [110]:
# Plain copy for now but we probably need much larger dimensions... 
class VariationalAutoEncoder(nn.Module):
    def __init__(self,D_in,H=50,H2=12,latent_dim=3):
        
        #Encoder
        super(VariationalAutoEncoder,self).__init__()
        self.linear1=nn.Linear(D_in,H)
        self.lin_bn1 = nn.BatchNorm1d(num_features=H)
        self.linear2=nn.Linear(H,H2)
        self.lin_bn2 = nn.BatchNorm1d(num_features=H2)
        self.linear3=nn.Linear(H2,H2)
        self.lin_bn3 = nn.BatchNorm1d(num_features=H2)
        
        # Latent vectors mu and sigma
        self.fc1 = nn.Linear(H2, latent_dim)
        self.bn1 = nn.BatchNorm1d(num_features=latent_dim)
        self.fc21 = nn.Linear(latent_dim, latent_dim)
        self.fc22 = nn.Linear(latent_dim, latent_dim)

        # Sampling vector
        self.fc3 = nn.Linear(latent_dim, latent_dim)
        self.fc_bn3 = nn.BatchNorm1d(latent_dim)
        self.fc4 = nn.Linear(latent_dim, H2)
        self.fc_bn4 = nn.BatchNorm1d(H2)
        
        # Decoder
        self.linear4=nn.Linear(H2,H2)
        self.lin_bn4 = nn.BatchNorm1d(num_features=H2)
        self.linear5=nn.Linear(H2,H)
        self.lin_bn5 = nn.BatchNorm1d(num_features=H)
        self.linear6=nn.Linear(H,D_in)
        self.lin_bn6 = nn.BatchNorm1d(num_features=D_in)
        
        self.relu = nn.ReLU()
        
    def encode(self, x):
        lin1 = self.relu(self.lin_bn1(self.linear1(x)))
        lin2 = self.relu(self.lin_bn2(self.linear2(lin1)))
        lin3 = self.relu(self.lin_bn3(self.linear3(lin2)))

        fc1 = F.relu(self.bn1(self.fc1(lin3)))

        r1 = self.fc21(fc1)
        r2 = self.fc22(fc1)
        
        return r1, r2
    
    def reparameterize(self, mu, logvar):
        if self.training:
            std = logvar.mul(0.5).exp_()
            eps = Variable(std.data.new(std.size()).normal_())
            return eps.mul(std).add_(mu)
        else:
            return mu
        
    def decode(self, z):
        fc3 = self.relu(self.fc_bn3(self.fc3(z)))
        fc4 = self.relu(self.fc_bn4(self.fc4(fc3)))

        lin4 = self.relu(self.lin_bn4(self.linear4(fc4)))
        lin5 = self.relu(self.lin_bn5(self.linear5(lin4)))
        return self.lin_bn6(self.linear6(lin5))


        
    def forward(self, x):
        mu, logvar = self.encode(x)
        z = self.reparameterize(mu, logvar)
        return self.decode(z), mu, logvar

In [111]:
class customLoss(nn.Module):
    def __init__(self):
        super(customLoss, self).__init__()
        self.mse_loss = nn.MSELoss(reduction="sum")
    
    def forward(self, x_recon, x, mu, logvar):
        loss_MSE = self.mse_loss(x_recon, x)
        loss_KLD = -0.5 * torch.sum(1 + logvar - mu.pow(2) - logvar.exp())

        return loss_MSE + loss_KLD

### Setup Hyperparameters

In [112]:
D_in = testdata_set.x.shape[1]
H = 1024
H2 = 128
latent_dim = 32
model = VariationalAutoEncoder(D_in, H, H2, latent_dim).to(device)
optimizer = optim.Adam(model.parameters(), lr=1e-3)
loss_mse = customLoss()

### Train Model

In [113]:
epochs = 50
log_interval = 5
val_losses = []
train_losses = []
test_losses = []

In [114]:
def train(epoch):
    model.train()
    train_loss = 0
    for batch_idx, data in enumerate(trainloader):
        data = data.to(device)
        optimizer.zero_grad()
        recon_batch, mu, logvar = model(data)
        loss = loss_mse(recon_batch, data, mu, logvar)
        loss.backward()
        train_loss += loss.item()
        optimizer.step()
    if epoch % log_interval == 0:        
        print('====> Epoch: {} Average training loss: {:.4f}'.format(
            epoch, train_loss / len(trainloader.dataset)))
        train_losses.append(train_loss / len(trainloader.dataset))

In [115]:
def test(epoch):
    with torch.no_grad():
        test_loss = 0
        for batch_idx, data in enumerate(testloader):
            data = data.to(device)
            optimizer.zero_grad()
            recon_batch, mu, logvar = model(data)
            loss = loss_mse(recon_batch, data, mu, logvar)
            test_loss += loss.item()
            if epoch % log_interval == 0:        
                print('====> Epoch: {} Average test loss: {:.4f}'.format(
                    epoch, test_loss / len(testloader.dataset)))
            test_losses.append(test_loss / len(testloader.dataset))

In [116]:
for epoch in range(1,epochs+1):
    train(epoch)
    test(epoch)

====> Epoch: 5 Average training loss: 1447.8418
====> Epoch: 5 Average test loss: 924.9739
====> Epoch: 5 Average test loss: 1137.5319
====> Epoch: 10 Average training loss: 1256.3754
====> Epoch: 10 Average test loss: 876.4720
====> Epoch: 10 Average test loss: 1083.0963
====> Epoch: 15 Average training loss: 1186.2606
====> Epoch: 15 Average test loss: 863.6194
====> Epoch: 15 Average test loss: 1068.5278
====> Epoch: 20 Average training loss: 1084.4003
====> Epoch: 20 Average test loss: 835.0503
====> Epoch: 20 Average test loss: 1033.6055
====> Epoch: 25 Average training loss: 1039.2670
====> Epoch: 25 Average test loss: 829.8318
====> Epoch: 25 Average test loss: 1021.1641
====> Epoch: 30 Average training loss: 943.2444
====> Epoch: 30 Average test loss: 810.0406
====> Epoch: 30 Average test loss: 999.5369
====> Epoch: 35 Average training loss: 906.3424
====> Epoch: 35 Average test loss: 789.5024
====> Epoch: 35 Average test loss: 976.2209
====> Epoch: 40 Average training loss: 86

In [117]:
with torch.no_grad():
    for batch_idx, data in enumerate(testloader):
        data = data.to(device)
        optimizer.zero_grad()
        recon_batch, mu, logvar = model(data)

In [118]:
scaler = trainloader.dataset.standardizer
recon_row = scaler.inverse_transform(recon_batch[0].cpu().numpy())
real_row = scaler.inverse_transform(testloader.dataset.x[0].cpu().numpy())

In [120]:
cols = frameEncodedSki.columns
df = pd.DataFrame(np.stack((recon_row, real_row)), columns = cols)
df

,amount1,amount2,amount3,amount4,amount5,amount6,amount7,amount8,amount9,amount10,...,ingredient17_nan,unit18_,unit18_nan,unit19_tablespoon,unit19_nan,ingredient18_Belgian endive,ingredient18_salt,ingredient18_nan,ingredient19_fresh chives,ingredient19_nan
0,-0.788912,3.675029,3.31487,15.415626,2.682431,16.542542,13.794868,36.561062,33.396538,23.504845,...,0.61772,3.669950e-01,0.64303,1.956798e-01,0.813185,2.196807e-01,2.543417e-01,0.638494,1.233099e-01,0.887837
1,1.000000,13.000000,14.00000,0.999999,-99.000000,-99.000000,-99.000000,-99.000000,-99.000000,-99.000000,...,1.00000,-1.596553e-10,1.00000,-7.982765e-11,1.000000,-7.982765e-11,-7.982765e-11,1.000000,-7.982765e-11,1.000000


### Draw random samples form latent space and generate new data

In [121]:
sigma = torch.exp(logvar/2)
# sample z from q
no_samples = 20
q = torch.distributions.Normal(mu.mean(axis=0), sigma.mean(axis=0))
z = q.rsample(sample_shape=torch.Size([no_samples]))

In [122]:
with torch.no_grad():
    pred = model.decode(z).cpu().numpy()

fake_data = scaler.inverse_transform(pred)
df_fake = pd.DataFrame(fake_data, columns = cols)
df_fake.head(10)

,amount1,amount2,amount3,amount4,amount5,amount6,amount7,amount8,amount9,amount10,...,ingredient17_nan,unit18_,unit18_nan,unit19_tablespoon,unit19_nan,ingredient18_Belgian endive,ingredient18_salt,ingredient18_nan,ingredient19_fresh chives,ingredient19_nan
0,2.622849,3.101445,1.952409,-3.286489,-6.917967,-7.127005,-15.173010,-14.809002,-24.895361,-78.218925,...,0.979895,0.010039,0.952298,-0.037629,1.002375,0.007946,-0.000974,0.961269,-0.028974,0.977558
1,4.403953,3.134321,1.241053,10.118871,19.466513,-3.896742,11.581699,-34.315571,-63.947655,-78.752411,...,0.971006,0.006920,1.031102,0.103126,0.875830,0.076690,-0.017688,0.978603,0.103127,0.910912
2,-2.518820,2.416171,-2.611117,-3.232556,2.008500,22.097136,2.193025,-27.620256,-54.224850,-27.822529,...,1.070561,-0.037227,1.076023,-0.135422,1.038255,-0.136600,-0.027875,1.086720,-0.069906,1.130483
3,7.957775,9.160497,7.616596,-11.201842,-14.368016,-25.096607,-45.559528,-68.370033,-112.043358,-112.468796,...,0.947477,0.009274,1.023925,-0.037405,1.039373,-0.086126,-0.030827,1.031077,-0.036118,0.998100
4,6.199277,5.819519,5.186251,-2.588488,1.645667,7.728661,-7.747354,-43.824097,-60.913914,-96.777321,...,0.966267,-0.050451,0.995892,-0.011612,1.010600,0.056957,-0.015309,0.990367,-0.036317,1.014485
5,0.502256,-2.485667,0.816783,6.313038,10.321754,4.827290,-30.013075,-45.991943,-45.128788,-65.235558,...,0.874049,0.094971,0.911520,0.049014,0.903028,0.093548,0.006839,0.864192,0.087480,0.961262
6,-5.219224,3.653841,10.473718,7.534138,5.946752,2.702822,19.315479,16.966284,23.584526,-10.512927,...,0.665753,0.148740,0.829901,0.196534,0.878596,0.123303,0.106880,0.873186,0.166522,0.853579
7,0.893000,12.032619,-1.976473,9.082327,20.789389,17.747080,2.395107,-78.145844,-85.909996,-105.570152,...,1.157428,-0.034139,1.132737,-0.037508,1.053732,-0.041370,-0.052619,1.109467,-0.022559,1.043887
8,11.666659,5.447656,0.370022,24.208273,27.054785,10.807321,-4.016907,-67.732475,-75.166107,-73.224808,...,0.677342,0.127520,0.816564,0.265002,0.782316,0.203032,0.031214,0.861969,0.274258,0.730640
9,9.728637,-6.130845,2.194805,4.020006,12.277905,-1.881008,15.756859,6.370161,-18.644173,-17.313393,...,0.987794,0.009841,0.974319,-0.014988,1.031296,-0.086293,0.048647,1.005895,-0.053739,0.992152


In [124]:
df_fake_stripped = df_fake.drop(columns=dropColumns)
df_fake_stripped_decoded = pd.DataFrame(data=enc.inverse_transform(df_fake_stripped), columns=frameStripped.columns)
df_fake_decoded = pd.concat([df_fake[dropColumns], df_fake_stripped_decoded], axis=1)
df_fake_decoded

,amount1,amount2,amount3,amount4,amount5,amount6,amount7,amount8,amount9,amount10,...,unit15,unit16,unit17,ingredient15,ingredient16,ingredient17,unit18,unit19,ingredient18,ingredient19
0,2.622849,3.101445,1.952409,-3.286489,-6.917967,-7.127005,-15.173010,-14.809002,-24.895361,-78.218925,...,NaN,NaN,NaN,fresh ground black pepper,NaN,NaN,NaN,NaN,NaN,NaN
1,4.403953,3.134321,1.241053,10.118871,19.466513,-3.896742,11.581699,-34.315571,-63.947655,-78.752411,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,-2.518820,2.416171,-2.611117,-3.232556,2.008500,22.097136,2.193025,-27.620256,-54.224850,-27.822529,...,NaN,NaN,NaN,fresh ground black pepper,NaN,NaN,NaN,NaN,NaN,NaN
3,7.957775,9.160497,7.616596,-11.201842,-14.368016,-25.096607,-45.559528,-68.370033,-112.043358,-112.468796,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,6.199277,5.819519,5.186251,-2.588488,1.645667,7.728661,-7.747354,-43.824097,-60.913914,-96.777321,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,0.502256,-2.485667,0.816783,6.313038,10.321754,4.827290,-30.013075,-45.991943,-45.128788,-65.235558,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,-5.219224,3.653841,10.473718,7.534138,5.946752,2.702822,19.315479,16.966284,23.584526,-10.512927,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7,0.893000,12.032619,-1.976473,9.082327,20.789389,17.747080,2.395107,-78.145844,-85.909996,-105.570152,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8,11.666659,5.447656,0.370022,24.208273,27.054785,10.807321,-4.016907,-67.732475,-75.166107,-73.224808,...,,NaN,NaN,fresh ground black pepper,NaN,NaN,NaN,NaN,NaN,NaN
9,9.728637,-6.130845,2.194805,4.020006,12.277905,-1.881008,15.756859,6.370161,-18.644173,-17.313393,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [131]:
class Ingredient:
    def __init__(self, amount, unit, ingredient) -> None:
        self.amount = amount
        self.unit = unit
        self.ingredient = ingredient

    def __repr__(self) -> str:
        return "Amount: " + str(self.amount) + "\n Unit: " + str(self.unit) + "\n Ingredient: " + str(self.ingredient) + "\n"

recipes = []
lenIngredients = int(len(df_fake_decoded.columns)/3)
for value in df_fake_decoded.values:
    ingredients = []
    for index in range(0,lenIngredients):
        ingredients.append(Ingredient(value[index], value[index+lenIngredients], value[index+2*lenIngredients]))
    recipes.append(ingredients)

print(recipes)

[[Amount: 2.622849225997925
 Unit: 
 Ingredient: nan, Amount: 3.101445198059082
 Unit: 
 Ingredient: nan, Amount: 1.9524093866348267
 Unit: 
 Ingredient: jarlsberg cheese, Amount: -3.2864890098571777
 Unit: tablespoon
 Ingredient: nan, Amount: -6.917967319488525
 Unit: 
 Ingredient: nan, Amount: -7.127004623413086
 Unit: 
 Ingredient: nan, Amount: -15.173009872436523
 Unit: 
 Ingredient: pine nuts, Amount: -14.809001922607422
 Unit: 
 Ingredient: nan, Amount: -24.895360946655273
 Unit: tablespoon
 Ingredient: nan, Amount: -78.21892547607422
 Unit: fennel seeds
 Ingredient: nan, Amount: -83.40631866455078
 Unit: dry breadcrumbs
 Ingredient: nan, Amount: -94.76396179199219
 Unit: boiling water
 Ingredient: nan, Amount: -86.74609375
 Unit: dried parsley
 Ingredient: fresh ground black pepper, Amount: -72.9424819946289
 Unit: romano cheese
 Ingredient: nan, Amount: -108.92888641357422
 Unit: Campbell's Cream of Mushroom Soup
 Ingredient: nan, Amount: -104.88788604736328
 Unit: parmesan che